<h2 id="tocheading">Table of Contents</h2>
<div id="toc"></div>

In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

In [2]:
import pickle
import random
import random
import spacy
import csv
import string
import os
import torch
import numpy as np
import pandas as pd
import spacy
from torch.utils.data import Dataset
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

## Part 1: Data Upload & Preprocessing
The datasets provided are already tokenized. Thus, without running the data through a tokenizer, we use pretrained word embeddings (e.g. fast-Text) to embed the tokens. 

#### Word Vectors

The web page for recommended word vector sets can be found here: https://fasttext.cc/docs/en/english-vectors.html wiki-news-300d-1M.vec from Mikolov et al (2018, Advances in Pre-Training Distributed Word Representations) 1 million word vectors trained on Wikipedia 2017, UMBC webbase corpus and statmt.org news dataset (16B tokens) is used in this assignment. 

In [3]:
import io

def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', 
                  newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        ## convert all maps to lists
        data[tokens[0]] = [*map(float, tokens[1:])]
    return data

In [4]:
## get the wiki word vectors
fname = "wiki-news-300d-1M.vec"
word_vectors = load_vectors(fname)

In [5]:
all_vocab_tokens = [*word_vectors.keys()]

In [6]:
print ("The number of unique tokens in the wiki news English vectors is " + str(len(all_vocab_tokens) ))

The number of unique tokens in the wiki news English vectors is 999994


#### Construct Table from Vocab Dict

In [7]:
word_vector_df = pd.DataFrame(word_vectors).T

In [8]:
table_lookup = np.array(word_vector_df)

In [9]:
def index_vocab(table_df):
    
    token_array = np.array([*table_df.index])
    num_index_array = np.array([*range(table_df.shape[0])])
    
    token2id = {}
    id2token = {}
    for i in [*range(len(token_array))]:
        token2id[token_array[i]] = num_index_array[i]
        id2token[num_index_array[i]] = token_array[i]

    return token2id, id2token

In [10]:
token2id_wiki, id2token_wiki = index_vocab(word_vector_df)

__Check for table correctness!__

Do token2id and id2token match each other?

In [11]:
token2id_wiki["Alberto"]

93141

In [12]:
id2token_wiki[93141]

'Alberto'

Does the table fit the initial word vector vocab?

In [13]:
all(word_vectors["Alberto"] == table_lookup[93141])==True

True

### Part 1.1: SNLI Dataset

In [14]:
label_dict = {"entailment":0,
             "neutral":1,
             "contradiction":2}

In [15]:
snli_train = pd.read_table("data/snli_train.tsv")
snli_val = pd.read_table("data/snli_val.tsv")

In [16]:
## get tokenized training data
snli_train["sentence1"] = snli_train["sentence1"].apply(lambda x: x.split(" "))
snli_train["sentence2"] = snli_train["sentence2"].apply(lambda x: x.split(" "))

In [17]:
## get labels
snli_train["label_num"] = snli_train["label"].apply(lambda x: label_dict[x])
snli_val["label_num"] = snli_val["label"].apply(lambda x: label_dict[x])

In [18]:
## get tokenized validation data
snli_val["sentence1"] = snli_val["sentence1"].apply(lambda x: x.split(" "))
snli_val["sentence2"] = snli_val["sentence2"].apply(lambda x: x.split(" "))

In [19]:
import pandas as pd

In [20]:
## get label arrays
snli_train_labels = np.array(snli_train["label_num"])
snli_val_labels = np.array(snli_val["label_num"])

In [21]:
snli_val.head(3)

,sentence1,sentence2,label,label_num
0,"[Three, women, on, a, stage, ,, one, wearing, ...","[There, are, two, women, standing, on, the, st...",contradiction,2
1,"[Four, people, sit, on, a, subway, two, read, ...","[Multiple, people, are, on, a, subway, togethe...",entailment,0
2,"[bicycles, stationed, while, a, group, of, peo...","[People, get, together, near, a, stand, of, bi...",entailment,0


### Part 1.2: MultiNLI Dataset

In [22]:
mnli_train = pd.read_table("data/mnli_train.tsv")
mnli_val = pd.read_table("data/mnli_val.tsv")

In [23]:
mnli_train.head(3)

,sentence1,sentence2,label,genre
0,and now that was in fifty one that 's forty ye...,It was already a problem forty years ago but n...,neutral,telephone
1,Jon could smell baked bread on the air and his...,Jon smelt food in the air and was hungry .,neutral,fiction
2,it will be like Italian basketball with the uh...,This type of Italian basketball is nothing lik...,contradiction,telephone


In [24]:
mnli_train["genre"].value_counts()

telephone     4270
slate         4026
travel        3985
government    3883
fiction       3836
Name: genre, dtype: int64

In [25]:
## get tokenized training data
mnli_train["sentence1"] = mnli_train["sentence1"].apply(lambda x: x.split(" "))
mnli_train["sentence2"] = mnli_train["sentence2"].apply(lambda x: x.split(" "))

In [26]:
## get tokenized validation data
mnli_val["sentence1"] = mnli_val["sentence1"].apply(lambda x: x.split(" "))
mnli_val["sentence2"] = mnli_val["sentence2"].apply(lambda x: x.split(" "))

In [27]:
## get labels
mnli_train["label_num"] = mnli_train["label"].apply(lambda x: label_dict[x])
mnli_val["label_num"] = mnli_val["label"].apply(lambda x: label_dict[x])

Get train and val datasets for each __MNLI genre__. 

In [28]:
## telephone
mnli_train_telephone = mnli_train[mnli_train["genre"]=="telephone"]
mnli_val_telephone = mnli_val[mnli_val["genre"]=="telephone"]
## slate
mnli_train_slate = mnli_train[mnli_train["genre"]=="slate"]
mnli_val_slate = mnli_val[mnli_val["genre"]=="slate"]
## travel
mnli_train_travel = mnli_train[mnli_train["genre"]=="travel"]
mnli_val_travel = mnli_val[mnli_val["genre"]=="travel"]
## government
mnli_train_government = mnli_train[mnli_train["genre"]=="government"]
mnli_val_government = mnli_val[mnli_val["genre"]=="government"]
## fiction
mnli_train_fiction = mnli_train[mnli_train["genre"]=="fiction"]
mnli_val_fiction = mnli_val[mnli_val["genre"]=="fiction"]

In [29]:
## get label arrays for each train and val dataset

## whole MNLI dataset
mnli_train_labels = np.array(pd.get_dummies(np.array(mnli_train["label_num"])))
mnli_val_labels = np.array(pd.get_dummies(np.array(mnli_val["label_num"])))
## telephone
mnli_train_tel_labels = np.array(pd.get_dummies(np.array(mnli_train_telephone["label_num"])))
mnli_val_tel_labels = np.array(pd.get_dummies(np.array(mnli_val_telephone["label_num"])))
## slate
mnli_train_slate_labels = np.array(pd.get_dummies(np.array(mnli_train_slate["label_num"])))
mnli_val_slate_labels = np.array(pd.get_dummies(np.array(mnli_val_slate["label_num"])))
## travel
mnli_train_travel_labels = np.array(pd.get_dummies(np.array(mnli_train_travel["label_num"])))
mnli_val_travel_labels = np.array(mnli_val_travel["label_num"])
## gov
mnli_train_gov_labels = np.array(pd.get_dummies(np.array(mnli_train_government["label_num"])))
mnli_val_gov_labels = np.array(pd.get_dummies(np.array(mnli_val_government["label_num"])))
## fiction
mnli_train_fiction_labels = np.array(pd.get_dummies(np.array(mnli_train_fiction["label_num"])))
mnli_val_fiction_labels = np.array(pd.get_dummies(np.array(mnli_val_fiction["label_num"])))

#### Data Loaders

In [30]:
## idx = token2id_wiki

def token2index_dataset(tokens_data,idx_dict=None):
    indices_data = []
    for tokens in tokens_data:
        ## get index list for each sentence.
        index_list = [idx_dict[token] if token in \
                      idx_dict else idx_dict["unk"] for token in tokens]
        indices_data.append(index_list)
    return indices_data

__Note:__ I am getting the indices for Sentence 1 and Sentence 2 separately (not concatenating them at first from the beginning) since, in hyperparameter search I want to try more than one ways of interacting the hidden representations of the two sentences. 

In [31]:
## get train and val indices for both datasets

## SNLI
snli_train_sentence1_indices = token2index_dataset([*snli_train["sentence1"]],idx_dict=token2id_wiki)
snli_train_sentence2_indices = token2index_dataset([*snli_train["sentence2"]],idx_dict=token2id_wiki)
snli_val_sentence1_indices = token2index_dataset([*snli_val["sentence1"]],idx_dict=token2id_wiki)
snli_val_sentence2_indices = token2index_dataset([*snli_val["sentence2"]],idx_dict=token2id_wiki)

## MNLI
mnli_train_sentence1_indices = token2index_dataset([*mnli_train["sentence1"]],idx_dict=token2id_wiki)
mnli_train_sentence2_indices = token2index_dataset([*mnli_train["sentence2"]],idx_dict=token2id_wiki)
mnli_val_sentence1_indices = token2index_dataset([*mnli_val["sentence1"]],idx_dict=token2id_wiki)
mnli_val_sentence2_indices = token2index_dataset([*mnli_val["sentence2"]],idx_dict=token2id_wiki)

In [32]:
## GENRES

## telephone
mnli_train_s1_tel_ix = token2index_dataset([*mnli_train_telephone["sentence1"]],idx_dict=token2id_wiki)
mnli_train_s2_tel_ix = token2index_dataset([*mnli_train_telephone["sentence2"]],idx_dict=token2id_wiki)
mnli_val_s1_tel_ix = token2index_dataset([*mnli_val_telephone["sentence1"]],idx_dict=token2id_wiki)
mnli_val_s2_tel_ix = token2index_dataset([*mnli_val_telephone["sentence2"]],idx_dict=token2id_wiki)
## slate
mnli_train_s1_slate_ix = token2index_dataset([*mnli_train_slate["sentence1"]],idx_dict=token2id_wiki)
mnli_train_s2_slate_ix = token2index_dataset([*mnli_train_slate["sentence2"]],idx_dict=token2id_wiki)
mnli_val_s1_slate_ix = token2index_dataset([*mnli_val_slate["sentence1"]],idx_dict=token2id_wiki)
mnli_val_s2_slate_ix = token2index_dataset([*mnli_val_slate["sentence2"]],idx_dict=token2id_wiki)
## travel
mnli_train_s1_travel_ix = token2index_dataset([*mnli_train_travel["sentence1"]],idx_dict=token2id_wiki)
mnli_train_s2_travel_ix = token2index_dataset([*mnli_train_travel["sentence2"]],idx_dict=token2id_wiki)
mnli_val_s1_travel_ix = token2index_dataset([*mnli_val_travel["sentence1"]],idx_dict=token2id_wiki)
mnli_val_s2_travel_ix = token2index_dataset([*mnli_val_travel["sentence2"]],idx_dict=token2id_wiki)
## gov
mnli_train_s1_gov_ix = token2index_dataset([*mnli_train_government["sentence1"]],idx_dict=token2id_wiki)
mnli_train_s2_gov_ix = token2index_dataset([*mnli_train_government["sentence2"]],idx_dict=token2id_wiki)
mnli_val_s1_gov_ix = token2index_dataset([*mnli_val_government["sentence1"]],idx_dict=token2id_wiki)
mnli_val_s2_gov_ix = token2index_dataset([*mnli_val_government["sentence2"]],idx_dict=token2id_wiki)
## fiction
mnli_train_s1_fiction_ix = token2index_dataset([*mnli_train_fiction["sentence1"]],idx_dict=token2id_wiki)
mnli_train_s2_fiction_ix = token2index_dataset([*mnli_train_fiction["sentence2"]],idx_dict=token2id_wiki)
mnli_val_s1_fiction_ix = token2index_dataset([*mnli_val_fiction["sentence1"]],idx_dict=token2id_wiki)
mnli_val_s2_fiction_ix = token2index_dataset([*mnli_val_fiction["sentence2"]],idx_dict=token2id_wiki)

Getting training and validation set __labels__ (targets) for both datasets. 

In [33]:
## SNLI
snli_train_labels = np.array(snli_train["label_num"])
snli_val_labels = np.array(snli_val["label_num"])

## MNLI
mnli_train_labels = np.array(mnli_train["label_num"])
mnli_val_labels = np.array(mnli_val["label_num"])

## GENRES

## telephone
mnli_train_tel_labels = np.array(mnli_train_telephone["label_num"])
mnli_val_tel_labels = np.array(mnli_val_telephone["label_num"])
## slate
mnli_train_slate_labels = np.array(mnli_train_slate["label_num"])
mnli_val_slate_labels = np.array(mnli_val_slate["label_num"])
## travel
mnli_train_travel_labels = np.array(mnli_train_travel["label_num"])
mnli_val_travel_labels = np.array(mnli_val_travel["label_num"])
## gov
mnli_train_gov_labels = np.array(mnli_train_government["label_num"])
mnli_val_gov_labels = np.array(mnli_val_government["label_num"])
## fiction
mnli_train_fiction_labels = np.array(mnli_train_fiction["label_num"])
mnli_val_fiction_labels = np.array(mnli_val_fiction["label_num"])

Function to get pretrained word embeddings from the table

In [34]:
## code taken from lab3

## SNLI
MAX_SENTENCE_LENGTH = 200
snli_train_targets = snli_train_labels
snli_val_targets = snli_val_labels

from torch.utils.data import Dataset

class SNLI_Dataset(Dataset):
    """
    Class that represents a train/validation/test dataset that's readable for PyTorch
    Note that this class inherits torch.utils.data.Dataset
    """
    
    def __init__(self, data_list, target_list):
        """
        @param data_list: list of newsgroup tokens 
        @param target_list: list of newsgroup targets 

        """
        self.data_list = data_list
        self.target_list = target_list
        assert (len(self.data_list) == len(self.target_list))

    def __len__(self):
        return len(self.data_list)
        
    def __getitem__(self, key):
        """
        Triggered when you call dataset[i]
        """
        
        token_idx = self.data_list[key][:MAX_SENTENCE_LENGTH]
        label = self.target_list[key]
        return [token_idx, len(token_idx), label]

def snli_func(batch):
    """
    Customized function for DataLoader that dynamically pads the batch so that all 
    data have the same length
    """
    data_list = []
    label_list = []
    length_list = []

    for datum in batch:
        label_list.append(datum[2])
        length_list.append(datum[1])
#         data_list.append(datum[0])
        
    # padding
    for datum in batch:
        padded_vec = np.pad(np.array(datum[0]), 
                                pad_width=((0,MAX_SENTENCE_LENGTH-datum[1])), 
                                mode="constant", constant_values=0)
        data_list.append(padded_vec)
        
    return [torch.from_numpy(np.array(data_list)), 
            torch.LongTensor(length_list), 
            torch.LongTensor(label_list)]

# create pytorch dataloader

# since I'm loading the sentences separately, I set shulle to False 


##### LABELS?

BATCH_SIZE = 32
snli_train_dataset_s1 = SNLI_Dataset(snli_train_sentence1_indices,snli_train_labels)
snli_train_dataset_s2 = SNLI_Dataset(snli_train_sentence2_indices,snli_train_labels)
snli_train_loader_s1 = torch.utils.data.DataLoader(dataset=snli_train_dataset_s1,
                                               batch_size=BATCH_SIZE,
                                               collate_fn=snli_func,
                                               shuffle=False)
snli_train_loader_s2 = torch.utils.data.DataLoader(dataset=snli_train_dataset_s2,
                                               batch_size=BATCH_SIZE,
                                               collate_fn=snli_func,
                                               shuffle=False)

snli_val_dataset_s1 = SNLI_Dataset(snli_val_sentence1_indices, snli_val_labels)
snli_val_dataset_s2 = SNLI_Dataset(snli_val_sentence2_indices, snli_val_labels)
snli_val_loader_s1 = torch.utils.data.DataLoader(dataset=snli_val_dataset_s1,
                                             batch_size=BATCH_SIZE,
                                             collate_fn=snli_func,
                                             shuffle=False)
snli_val_loader_s2 = torch.utils.data.DataLoader(dataset=snli_val_dataset_s2,
                                             batch_size=BATCH_SIZE,
                                             collate_fn=snli_func,
                                             shuffle=False)

I will load two strings separately, and get the hidden representations as they're output from the encoder - again separately, then interact them to get the softmax.

In [35]:
## TODO

## code taken from lab3
## mnli
MAX_SENTENCE_LENGTH = 200
mnli_train_targets = mnli_train_labels
mnli_val_targets = mnli_val_labels

from torch.utils.data import Dataset

class MNLI_Dataset(Dataset):
    """
    Class that represents a train/validation/test dataset that's readable for PyTorch
    Note that this class inherits torch.utils.data.Dataset
    """
    
    def __init__(self, data_list, target_list):
        """
        @param data_list: list of newsgroup tokens 
        @param target_list: list of newsgroup targets 

        """
        self.data_list = data_list
        self.target_list = target_list
        assert (len(self.data_list) == len(self.target_list))

    def __len__(self):
        return len(self.data_list)
        
    def __getitem__(self, key):
        """
        Triggered when you call dataset[i]
        """
        
        token_idx = self.data_list[key][:MAX_SENTENCE_LENGTH]
        label = self.target_list[key]
        return [token_idx, len(token_idx), label]

def mnli_func(batch):
    """
    Customized function for DataLoader that dynamically pads the batch so that all 
    data have the same length
    """
    data_list = []
    label_list = []
    length_list = []

    for datum in batch:
        label_list.append(datum[2])
        length_list.append(datum[1])
        
    # padding
    for datum in batch:
        padded_vec = np.pad(np.array(datum[0]), 
                                pad_width=((0,MAX_SENTENCE_LENGTH-datum[1])), 
                                mode="constant", constant_values=0)
            
        data_list.append(padded_vec)
        
        
        
    return [torch.from_numpy(np.array(data_list)), 
            torch.LongTensor(length_list), 
            torch.LongTensor(label_list)]

# create pytorch dataloader

BATCH_SIZE = 32
mnli_train_dataset_s1 = MNLI_Dataset(mnli_train_sentence1_indices,mnli_train_labels)
mnli_train_loader_s1 = torch.utils.data.DataLoader(dataset=mnli_train_dataset_s1,
                                               batch_size=BATCH_SIZE,
                                               collate_fn=mnli_func,
                                               shuffle=False)
mnli_train_dataset_s2 = MNLI_Dataset(mnli_train_sentence2_indices,mnli_train_labels)
mnli_train_loader_s2 = torch.utils.data.DataLoader(dataset=mnli_train_dataset_s2,
                                               batch_size=BATCH_SIZE,
                                               collate_fn=mnli_func,
                                               shuffle=False)


mnli_val_dataset_s1 = MNLI_Dataset(mnli_val_sentence1_indices, mnli_val_labels)
mnli_val_loader_s1 = torch.utils.data.DataLoader(dataset=mnli_val_dataset_s1,
                                             batch_size=BATCH_SIZE,
                                             collate_fn=mnli_func,
                                             shuffle=False)

mnli_val_dataset_s2 = MNLI_Dataset(mnli_val_sentence2_indices, mnli_val_labels)
mnli_val_loader_s2 = torch.utils.data.DataLoader(dataset=mnli_val_dataset_s2,
                                             batch_size=BATCH_SIZE,
                                             collate_fn=mnli_func,
                                             shuffle=False)

In [36]:
(" ").join([id2token_wiki[x] for x in [*mnli_train_dataset_s1][0][0]])

"and now that was in fifty one that 's forty years ago that it was already a problem so it 's now uh"

In [37]:
(" ").join([id2token_wiki[x] for x in [*mnli_train_dataset_s2][0][0]])

"It was already a problem forty years ago but now it 's ten times worse !"

In [38]:
# [*mnli_train_loader_s2][29]

## Part 2: Model

The model is trained on SNLI training set. The best model is chosen using SNLI validation set, then the best model is evaluated on each genre in MultiNLI validation set. 

We will use an encoder (either a CNN or an RNN) to map each string of text (hypothesis and premise) to a fixed-dimension vector representation. 

- We will interact the two hidden representations and output a __3-class softmax__. 

- To keep things simple, we will simply __concatenate__ the two representations, and feed them through a network of __2 fully-connected layers__. 

- For the encoder, we want the following:

### Part 2.1: CNN
For the CNN, 
- A 2-layer 1-D convolutional network with ReLU activations 
- A max-pool at the end to compress the hidden representation into a single vector.

In [47]:
# modify to accept hard coded arguments
# batch_size = 8
# batch_size = 16
epochs = 20
no_cuda = False
log_interval = 1

cuda = not no_cuda and torch.cuda.is_available()
# cuda = False

seed = 1
torch.manual_seed(seed)

# device = torch.device("cuda" if args.cuda else "cpu")
device = torch.device("cuda" if cuda else "cpu")

# kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}

In [60]:
max_train_length = max([snli_train_dataset_s1[x][1] for x in range(len(snli_train_dataset_s1))])

In [61]:
# max([snli_train_dataset_s2[x][1] for x in range(len(snli_train_dataset_s2))])

In [62]:
max_val_length = max([snli_val_dataset_s1[x][1] for x in range(len(snli_val_dataset_s1))])

In [63]:
# max([snli_val_dataset_s2[x][1] for x in range(len(snli_val_dataset_s2))])

In [49]:
# train_dataset = [*snli_train_dataset]

class CNN(nn.Module):
    
    def __init__(self,emb_size,
                 hidden_size,
                 hidden_size_2,
                 num_classes,
                 vocab_size,
                 kernel_size,
                 padding,
                 stride,
                 percent_dropout,
                 interaction_type):

        super(CNN, self).__init__()

        self.emb_size = emb_size
        self.kernel_size = kernel_size
        self.padding = padding
        self.hidden_size = hidden_size
        self.hidden_size_2 = hidden_size_2
        self.stride = stride
        self.interaction = interaction_type

        ## use pretrained wiki embeddings
        wiki_embed_table = torch.FloatTensor(table_lookup).to(device)
        embedding = nn.Embedding.from_pretrained(wiki_embed_table)
        self.embedding = embedding
        
        ## 2 1d convolutional layers
        ## conv1
        self.conv1 = nn.Conv1d(emb_size, hidden_size,
                               kernel_size=self.kernel_size, 
                               padding=self.padding)
        ## conv2
        self.conv2 = nn.Conv1d(hidden_size, hidden_size_2,
                               kernel_size=self.kernel_size, 
                               padding=self.padding)
        
        ## ReLU activations
        self.ReLU1 = nn.ReLU()
        self.ReLU2 = nn.ReLU()
        
        ## dropout
        self.dropout = nn.Dropout(percent_dropout)
        
        ## Fully Connected Layers for Concatenated 
        ## Hidden Representations
        
        ####
        ####
        ####
        hidden_out = int(3+(hidden_size_2-self.kernel_size + 2*self.padding)/self.stride)

        if self.interaction == "concat":
            self.fc1 = nn.Linear(hidden_out*2, hidden_size_2)
            self.fc2 = nn.Linear(hidden_size_2, 3)
        else: 
            ## no need to multiply hidden size by 2
            self.fc1 = nn.Linear(hidden_out, hidden_size_2)
            self.fc2 = nn.Linear(hidden_size_2, 3)
        

    def forward(self, x1, x2, lengths1, lengths2):
        batch_size_1, seq_len_1 = x1.size()
        batch_size_2, seq_len_2 = x2.size()
        
        ## change embedding layer with
#         embeds_1 = torch.FloatTensor(self.embedding(x1)).to(device)
#         embeds_2 = torch.FloatTensor(self.embedding(x2)).to(device)

        embeds_1 = []
        embeds_2 = []
        for arr in [*range(len(x1))]: 
            input = torch.cuda.LongTensor(x1[arr][:int(lengths1[arr])].cpu().numpy()).to(device)
            embed = self.embedding(input).to(device)
            ## pad again
            length_to_pad = MAX_SENTENCE_LENGTH - embed.size()[0]
            embed = np.vstack((embed.cpu().numpy(),np.zeros((length_to_pad,300))))
            embeds_1.append(embed)
            
        embeds_1 = torch.FloatTensor(embeds_1).to(device)
            
        for arr in [*range(len(x2))]: 
            input = torch.cuda.LongTensor(x2[arr][:int(lengths2[arr])].cpu().numpy()).to(device)
            embed = self.embedding(input).to(device)
            ## pad again
            length_to_pad = MAX_SENTENCE_LENGTH - embed.size()[0]
            embed = np.vstack((embed.cpu().numpy(),np.zeros((length_to_pad,300))))
            embeds_2.append(embed)

        embeds_2 = torch.FloatTensor(embeds_2).to(device)

        ## FIRST 1D CONVOLUTION
        ## sentence 1 - conv1
        s1_conv1 = self.conv1(torch.transpose(embeds_1, 1, 2)).transpose(1,2)
        ## sentence 2 - conv 1
        s2_conv1 = self.conv1(torch.transpose(embeds_2, 1, 2)).transpose(1,2)

        ## FIRST RELU
        ## sentence 1 - ReLU1
        s1_ReLU1 = self.ReLU1(s1_conv1)
        ## sentence 2 - ReLU1
        s2_ReLU1 = self.ReLU1(s2_conv1)

        ## SECOND 1D CONVOLUTION
        ## sentence 1 - conv2
        s1_conv2 = self.conv2(s1_ReLU1.transpose(1,2)).transpose(1,2)
        ## sentence 2 - conv2
        s2_conv2 = self.conv2(s2_ReLU1.transpose(1,2)).transpose(1,2)
        
        ## SECOND RELU
        ## sentence 1 - ReLU2
        s1_ReLU2 = self.ReLU2(s1_conv2)
        ## sentence 2 - ReLU2
        s2_ReLU2 = self.ReLU2(s2_conv2)
        
        ## MAX-POOL - dropout
        s1_hidden = torch.max(self.dropout(s1_ReLU2),1)
        s2_hidden = torch.max(self.dropout(s2_ReLU2),1)

        ## INTERACTION
        if self.interaction == "concat":
            ## CONCATENATION
            hidden = torch.cat((s1_hidden[0],s2_hidden[0]),1)
        elif self.interaction == "mul":
            ## ELEM-WISE MULTP.
            hidden = s1_hidden[0]*s2_hidden[0]
        elif self.interaction == "subtract":
            ## SUBTRACTION
            hidden = s1_hidden[0] - s2_hidden[0]

        ## FC LAYERS
        hidden = self.fc1(hidden)
        hidden = self.ReLU2(hidden)
        hidden = self.fc2(hidden)

        ## SOFTMAX
        out = F.softmax(hidden)
        return out

In [64]:
import numpy as np

#### Initial Training

In [135]:
wiki_embed_table = torch.FloatTensor(table_lookup).to(device)

In [72]:
model = CNN(emb_size=300, 
            hidden_size=512,
            hidden_size_2 = 128,
            num_classes=3, 
            vocab_size=wiki_embed_table.size()[0],
            kernel_size=3,
            padding=1,
            stride=1,
            percent_dropout=0.1,
            interaction_type="concat").to(device)

In [67]:
# torch.save(model.state_dict(),"model")

In [68]:
# torch.load("model")

In [69]:
learning_rate = 1e-3
num_epochs = 10 # number epoch to train

In [70]:
criterion = torch.nn.NLLLoss().cuda()
# optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [71]:
# [*enumerate(zip(snli_train_loader_s1,snli_train_loader_s2))][0]

In [74]:
def test_model(loader1, loader2, model):
    """
    Takes as input validation loaders as loader1 and loader2, 
    and the model fitted to the training set.
    """
    correct = 0
    total = 0
    
    model.eval()
    for batch_idx, ([data_s1, lengths_s1, labels], 
    [data_s2, lengths_s2, labels]) in enumerate(zip(loader1,
                                                 loader2)):
        ## validation data, length tensors, and val labels 
        ## to device
        data_s1 = data_s1.to(device)
        data_s2 = data_s2.to(device)
        lengths_s1 = lengths_s1.to(device)
        lengths_s2 = lengths_s2.to(device)
        labels = labels.to(device)
        
        outputs = model(data_s1, data_s2, lengths_s1, lengths_s2).to(device)
        predicted = torch.max(outputs, 1)[1]
#         print ("preds="+str(predicted))
        
        total += labels.size(0)
        correct += np.sum((predicted.cpu().detach().numpy() == labels.cpu().numpy()).astype(int))

    return (correct / total)

total_step = len(snli_train_loader_s1)
# num_epochs = 10
num_epochs = 4
loss_hist = []
val_accuracy = []
train_accuracy = []

for epoch in range(num_epochs):
#     train(epoch)
    for batch_idx, ([data_s1, lengths_s1, labels], 
    [data_s2, lengths_s2, labels]) in enumerate(zip(snli_train_loader_s1,
                                                 snli_train_loader_s2)):
        
        optimizer.zero_grad()
        
        data_s1 = data_s1.to(device)
        data_s2 = data_s2.to(device)
        lengths_s1 = lengths_s1.to(device)
        lengths_s2 = lengths_s2.to(device)
        labels = labels.to(device)
#         print ("Data transfer complete.")
#         model.train()
        outputs = model(data_s1,data_s2,lengths_s1,lengths_s2)
        predicted = torch.max(outputs, 1)[1]
        loss = criterion(outputs, labels).to(device)
        loss.cuda().backward()
        optimizer.step()
        
        # Forward pass
        
        ## Compute accuracy
        
        train_acc = np.sum((predicted.cpu().detach().numpy() == labels.cpu().numpy()).\
                           astype(int))/predicted.size(0)
            
        if batch_idx % 30 == 0:
            train_accuracy.append(train_acc)
            loss_hist.append(loss)
            # validate
            ## test model with validation loaders
            val_acc = test_model(snli_val_loader_s1,snli_val_loader_s2, model)
            val_accuracy.append(val_acc)
            
        if batch_idx > 0 and batch_idx % 600 == 0:
            print('Epoch: [{}/{}], Step: [{}/{}], Validation Acc: {}'.format(
                       epoch+1, num_epochs, batch_idx+1, len(snli_train_loader_s1), 
                val_accuracy[-1]))
            
            ## Plot validation accuracy
            plt.figure(figsize=(20,10))
            plt.plot(val_accuracy,
                     color="crimson",linewidth=3,alpha=0.6,marker="o")
            plt.rcParams["font.size"] = 16
            plt.title("Validation Accuracy")
            plt.xlabel("Iterations")
            plt.ylabel("Accuracy")
            plt.show()      
            
        if batch_idx % 3000 == 0:
            ## Plot training accuracy
            plt.figure(figsize=(20,10))
            plt.plot(train_accuracy,
                     color="b",linewidth=3,alpha=0.6,marker="o")
            plt.rcParams["font.size"] = 16
            plt.title("Training Accuracy")
            plt.xlabel("Iterations")
            plt.ylabel("Accuracy")
            plt.show()
            print ("Cross Entropy = "+str(loss))
            torch.save(model.state_dict(),"model")
            
                  

#### Hyperparameter Search

The hyperparameters included in the hyperparameter search space are;

- Learning rate of the optimizer
- The size of the hidden dimension of the CNN,
- The kernel size of the CNN,
- Experiment with different ways of interacting the two encoded sentences (concatenation, element-wise multiplication, outer multiplication etc)
- Dropout prob.

In [75]:
import itertools

params = [[1e-4, 1e-3,1e-2], ## learning rates
         [256,512], ## hidden dimension
         [128,64,32], ## hidden dimension 2
         [0.1,0.3,0.5], ## dropout
         [5],
#          [3,5,11], ## kernel size
         ["concat","subtract"] ## interaction type
         ]

params = [*itertools.product(*params)]

In [80]:
## cnn hyperparameter search

param_losses = {}

for param_set in params[40:]:
    print ("Parameter Set: "+str(param_set))
    ## INITIALIZE VALIDATION ACCURACY LIST
    param_losses[param_set] = []
    
    model = CNN(emb_size=300, 
            hidden_size=param_set[1],
            hidden_size_2 = param_set[2],
            num_classes=3, 
            vocab_size=wiki_embed_table.size()[0],
            kernel_size=param_set[4],
            padding=1,
            stride=1,
            percent_dropout=param_set[3],
            interaction_type=param_set[5]).to(device)
    
    # criterion = torch.nn.CrossEntropyLoss()
    criterion = torch.nn.NLLLoss().cuda()
    # optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    optimizer = torch.optim.Adam(model.parameters(), 
                                 lr=param_set[0],
                                 weight_decay=1e-4)
    
    num_epochs = 5
    
    for epoch in range(num_epochs):
        print ("Epoch = " + str(epoch))
        
        for batch_idx, ([data_s1, lengths_s1, labels], 
        [data_s2, lengths_s2, labels]) in enumerate(zip(snli_train_loader_s1,
                                                     snli_train_loader_s2)):

            optimizer.zero_grad()

            data_s1 = data_s1.to(device)
            data_s2 = data_s2.to(device)
            lengths_s1 = lengths_s1.to(device)
            lengths_s2 = lengths_s2.to(device)
            labels = labels.to(device)
            outputs = model(data_s1,data_s2,lengths_s1,lengths_s2)
            predicted = torch.max(outputs, 1)[1]
            loss = criterion(outputs, labels).to(device)
            loss.cuda().backward()
            optimizer.step()

            if batch_idx % 1000 == 0:
                # validate
                ## test model with validation loaders
                val_acc = test_model(snli_val_loader_s1,snli_val_loader_s2, model)
                param_losses[param_set].append(val_acc)
        
            if epoch == 4 and batch_idx > 0 and batch_idx % 3000 == 0:
                print ("Validation Accuracy = " + str(val_acc))
                pd.DataFrame(param_losses).to_csv("param_losses_cnn_7.csv")

Parameter Set: (0.001, 256, 128, 0.5, 5, 'concat')
Epoch = 0


/home/adc563/pytorch-cpu/py3.6.3/lib/python3.6/site-packages/ipykernel_launcher.py:140: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch = 1
Epoch = 2
Epoch = 3
Epoch = 4
Validation Accuracy = 0.602
Parameter Set: (0.001, 256, 128, 0.5, 5, 'subtract')
Epoch = 0
Epoch = 1
Epoch = 2
Epoch = 3
Epoch = 4
Validation Accuracy = 0.528
Parameter Set: (0.001, 256, 64, 0.1, 5, 'concat')
Epoch = 0
Epoch = 1
Epoch = 2
Epoch = 3
Epoch = 4
Validation Accuracy = 0.6
Parameter Set: (0.001, 256, 64, 0.1, 5, 'subtract')
Epoch = 0
Epoch = 1
Epoch = 2
Epoch = 3
Epoch = 4
Validation Accuracy = 0.575
Parameter Set: (0.001, 256, 64, 0.3, 5, 'concat')
Epoch = 0
Epoch = 1
Epoch = 2
Epoch = 3
Epoch = 4
Validation Accuracy = 0.616
Parameter Set: (0.001, 256, 64, 0.3, 5, 'subtract')
Epoch = 0
Epoch = 1
Epoch = 2
Epoch = 3
Epoch = 4
Validation Accuracy = 0.574
Parameter Set: (0.001, 256, 64, 0.5, 5, 'concat')
Epoch = 0
Epoch = 1
Epoch = 2
Epoch = 3
Epoch = 4
Validation Accuracy = 0.608
Parameter Set: (0.001, 256, 64, 0.5, 5, 'subtract')
Epoch = 0
Epoch = 1
Epoch = 2
Epoch = 3
Epoch = 4
Validation Accuracy = 0.592
Parameter Set: (0.001, 256, 3

KeyboardInterrupt: 

## Part 3: Test Performance

The best model was evaluated on each Multi-NLI genre.

In [40]:
## get the best performing model from validation
import numpy as np
import pandas as pd 

validation_dfs = []

for i in range(5,8):
    validation_dfs.append(pd.\
    DataFrame(pd.read_csv("param_losses_cnn_" \
                          + str(i)+".csv",header=None)).drop(0, 1))
    
validation_df = pd.DataFrame(pd.concat(validation_dfs,1).T)
validation_df.columns=["lr","hidden","hidden_2",
                                     "dropout","kernel","interaction"]+[*range(1,21)]

validation_df.index = [*range(len(validation_df))]

validation_ = pd.DataFrame(validation_df).dropna(0)

In [41]:
validation_.head(3)

,lr,hidden,hidden_2,dropout,kernel,interaction,1,2,3,4,...,11,12,13,14,15,16,17,18,19,20
0,0.0001,256,32,0.1,5,concat,0.338,0.535,0.558,0.579,...,0.587,0.583,0.577,0.587,0.584,0.592,0.58,0.591,0.583,0.593
1,0.0001,256,32,0.1,5,subtract,0.331,0.528,0.559,0.574,...,0.592,0.602,0.596,0.586,0.598,0.599,0.599,0.593,0.6,0.604
2,0.0001,256,32,0.3,5,concat,0.331,0.535,0.567,0.582,...,0.586,0.588,0.578,0.58,0.599,0.594,0.594,0.586,0.601,0.601


In [42]:
validation_["max_val_acc"] = [max(validation_[[*range(1,21)]].iloc[i]) for i in range(len(validation_))]

In [43]:
validation_.head(3)

,lr,hidden,hidden_2,dropout,kernel,interaction,1,2,3,4,...,12,13,14,15,16,17,18,19,20,max_val_acc
0,0.0001,256,32,0.1,5,concat,0.338,0.535,0.558,0.579,...,0.583,0.577,0.587,0.584,0.592,0.58,0.591,0.583,0.593,0.593
1,0.0001,256,32,0.1,5,subtract,0.331,0.528,0.559,0.574,...,0.602,0.596,0.586,0.598,0.599,0.599,0.593,0.6,0.604,0.604
2,0.0001,256,32,0.3,5,concat,0.331,0.535,0.567,0.582,...,0.588,0.578,0.58,0.599,0.594,0.594,0.586,0.601,0.601,0.601


In [44]:
np.where(np.array(validation_["max_val_acc"])==max(validation_["max_val_acc"]))

(array([35]),)

In [45]:
best_model_series = validation_.iloc[35] 

lr = float(best_model_series["lr"])
hidden_size = int(best_model_series["hidden"])
hidden_size_2 = int(best_model_series["hidden_2"])
dropout_pr = float(best_model_series["dropout"])
kernel_size = int(best_model_series["kernel"])
interaction_type = best_model_series["interaction"]

In [50]:
wiki_embed_table = torch.FloatTensor(table_lookup).to(device)

model = CNN(emb_size=300, 
            hidden_size=hidden_size,
            hidden_size_2 = hidden_size_2,
            num_classes=3, 
            vocab_size=wiki_embed_table.size()[0],
            kernel_size=kernel_size,
            padding=1,
            stride=1,
            percent_dropout=dropout_pr,
            interaction_type=interaction_type).to(device)

learning_rate = lr
num_epochs = 4 # number epoch to train

criterion = torch.nn.NLLLoss().cuda()
# optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

def test_model(loader1, loader2, model):
    """
    Takes as input validation loaders as loader1 and loader2, 
    and the model fitted to the training set.
    """
    correct = 0
    total = 0
    
    model.eval()
    for batch_idx, ([data_s1, lengths_s1, labels], 
    [data_s2, lengths_s2, labels]) in enumerate(zip(loader1,
                                                 loader2)):
        ## validation data, length tensors, and val labels 
        ## to device
        data_s1 = data_s1.to(device)
        data_s2 = data_s2.to(device)
        lengths_s1 = lengths_s1.to(device)
        lengths_s2 = lengths_s2.to(device)
        labels = labels.to(device)
        
        outputs = model(data_s1, data_s2, lengths_s1, lengths_s2).to(device)
        predicted = torch.max(outputs, 1)[1]
#         print ("preds="+str(predicted))
        
        total += labels.size(0)
        correct += np.sum((predicted.cpu().detach().numpy() == labels.cpu().numpy()).astype(int))

    return (correct / total)

In [51]:
mnli_val["genre"].value_counts()

government    1016
telephone     1005
slate         1002
fiction        995
travel         982
Name: genre, dtype: int64

In [53]:
## train selected model

num_epochs = 5

for epoch in range(num_epochs):
    print ("Epoch = "+str(epoch))
    for batch_idx, ([data_s1, lengths_s1, labels], 
    [data_s2, lengths_s2, labels]) in enumerate(zip(snli_train_loader_s1,
                                                 snli_train_loader_s2)):
        
        optimizer.zero_grad()
        
        data_s1 = data_s1.to(device)
        data_s2 = data_s2.to(device)
        lengths_s1 = lengths_s1.to(device)
        lengths_s2 = lengths_s2.to(device)
        labels = labels.to(device)
#         print ("Data transfer complete.")
#         model.train()
        outputs = model(data_s1,data_s2,lengths_s1,lengths_s2)
        predicted = torch.max(outputs, 1)[1]
        loss = criterion(outputs, labels).to(device)
        loss.cuda().backward()
        optimizer.step()

Epoch = 0


/home/adc563/pytorch-cpu/py3.6.3/lib/python3.6/site-packages/ipykernel_launcher.py:140: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


#### Government

In [148]:
## data loaders
BATCH_SIZE = mnli_val.shape[0]

mnli_val_dataset_s1 = MNLI_Dataset(mnli_val_s1_gov_ix, mnli_val_gov_labels)

mnli_val_loader_s1 = torch.utils.data.DataLoader(dataset=mnli_val_dataset_s1,
                                             batch_size=BATCH_SIZE,
                                             collate_fn=mnli_func,
                                             shuffle=False)

mnli_val_dataset_s2 = MNLI_Dataset(mnli_val_s2_gov_ix, mnli_val_gov_labels)

mnli_val_loader_s2 = torch.utils.data.DataLoader(dataset=mnli_val_dataset_s2,
                                             batch_size=BATCH_SIZE,
                                             collate_fn=mnli_func,
                                             shuffle=False)

test_accuracy = test_model(mnli_val_loader_s1,
                           mnli_val_loader_s2,
                           model)

print ("Government genre test accuracy is "+ str(test_accuracy))

Government genre test accuracy is 0.4734251968503937


/home/adc563/pytorch-cpu/py3.6.3/lib/python3.6/site-packages/ipykernel_launcher.py:140: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


#### Telephone

In [149]:
## data loaders
BATCH_SIZE = mnli_val.shape[0]

mnli_val_dataset_s1 = MNLI_Dataset(mnli_val_s1_tel_ix, mnli_val_tel_labels)

mnli_val_loader_s1 = torch.utils.data.DataLoader(dataset=mnli_val_dataset_s1,
                                             batch_size=BATCH_SIZE,
                                             collate_fn=mnli_func,
                                             shuffle=False)

mnli_val_dataset_s2 = MNLI_Dataset(mnli_val_s2_tel_ix, mnli_val_tel_labels)

mnli_val_loader_s2 = torch.utils.data.DataLoader(dataset=mnli_val_dataset_s2,
                                             batch_size=BATCH_SIZE,
                                             collate_fn=mnli_func,
                                             shuffle=False)

test_accuracy = test_model(mnli_val_loader_s1,
                           mnli_val_loader_s2,
                           model)

print ("Telephone genre test accuracy is "+ str(test_accuracy))

Telephone genre test accuracy is 0.4766169154228856


/home/adc563/pytorch-cpu/py3.6.3/lib/python3.6/site-packages/ipykernel_launcher.py:140: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


#### Slate

In [150]:
## data loaders
BATCH_SIZE = mnli_val.shape[0]

mnli_val_dataset_s1 = MNLI_Dataset(mnli_val_s1_slate_ix, mnli_val_slate_labels)

mnli_val_loader_s1 = torch.utils.data.DataLoader(dataset=mnli_val_dataset_s1,
                                             batch_size=BATCH_SIZE,
                                             collate_fn=mnli_func,
                                             shuffle=False)

mnli_val_dataset_s2 = MNLI_Dataset(mnli_val_s2_slate_ix, mnli_val_slate_labels)

mnli_val_loader_s2 = torch.utils.data.DataLoader(dataset=mnli_val_dataset_s2,
                                             batch_size=BATCH_SIZE,
                                             collate_fn=mnli_func,
                                             shuffle=False)

test_accuracy = test_model(mnli_val_loader_s1,
                           mnli_val_loader_s2,
                           model)

print ("Slate genre test accuracy is "+ str(test_accuracy))

Slate genre test accuracy is 0.40119760479041916


/home/adc563/pytorch-cpu/py3.6.3/lib/python3.6/site-packages/ipykernel_launcher.py:140: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


#### Fiction

In [151]:
## data loaders
BATCH_SIZE = mnli_val.shape[0]

mnli_val_dataset_s1 = MNLI_Dataset(mnli_val_s1_fiction_ix, mnli_val_fiction_labels)

mnli_val_loader_s1 = torch.utils.data.DataLoader(dataset=mnli_val_dataset_s1,
                                             batch_size=BATCH_SIZE,
                                             collate_fn=mnli_func,
                                             shuffle=False)

mnli_val_dataset_s2 = MNLI_Dataset(mnli_val_s2_fiction_ix, mnli_val_fiction_labels)

mnli_val_loader_s2 = torch.utils.data.DataLoader(dataset=mnli_val_dataset_s2,
                                             batch_size=BATCH_SIZE,
                                             collate_fn=mnli_func,
                                             shuffle=False)

test_accuracy = test_model(mnli_val_loader_s1,
                           mnli_val_loader_s2,
                           model)

print ("Fiction genre test accuracy is "+ str(test_accuracy))

Fiction genre test accuracy is 0.4542713567839196


/home/adc563/pytorch-cpu/py3.6.3/lib/python3.6/site-packages/ipykernel_launcher.py:140: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


#### Travel

In [152]:
## data loaders
BATCH_SIZE = mnli_val.shape[0]

mnli_val_dataset_s1 = MNLI_Dataset(mnli_val_s1_travel_ix, mnli_val_travel_labels)

mnli_val_loader_s1 = torch.utils.data.DataLoader(dataset=mnli_val_dataset_s1,
                                             batch_size=BATCH_SIZE,
                                             collate_fn=mnli_func,
                                             shuffle=False)

mnli_val_dataset_s2 = MNLI_Dataset(mnli_val_s2_travel_ix, mnli_val_travel_labels)

mnli_val_loader_s2 = torch.utils.data.DataLoader(dataset=mnli_val_dataset_s2,
                                             batch_size=BATCH_SIZE,
                                             collate_fn=mnli_func,
                                             shuffle=False)

test_accuracy = test_model(mnli_val_loader_s1,
                           mnli_val_loader_s2,
                           model)

print ("Travel genre test accuracy is "+ str(test_accuracy))

Travel genre test accuracy is 0.4460285132382892


/home/adc563/pytorch-cpu/py3.6.3/lib/python3.6/site-packages/ipykernel_launcher.py:140: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


### Part 4: Misclassification Examples

Just changed the test model to print out predicted values and original labels.

In [76]:
def test_model(loader1, loader2, model):
    """
    Takes as input validation loaders as loader1 and loader2, 
    and the model fitted to the training set.
    """
    correct = 0
    total = 0
    
    model.eval()
    for batch_idx, ([data_s1, lengths_s1, labels], 
    [data_s2, lengths_s2, labels]) in enumerate(zip(loader1,
                                                 loader2)):
        ## validation data, length tensors, and val labels 
        ## to device
        data_s1 = data_s1.to(device)
        data_s2 = data_s2.to(device)
        lengths_s1 = lengths_s1.to(device)
        lengths_s2 = lengths_s2.to(device)
        labels = labels.to(device)
        print ("labels = "+str(list(labels.cpu().numpy())))
        
        outputs = model(data_s1, data_s2, lengths_s1, lengths_s2).to(device)
        predicted = torch.max(outputs, 1)[1]
        print ("predicted = "+str(list(predicted.cpu().numpy())))
#         print ("preds="+str(predicted))
        
        total += labels.size(0)
        correct += np.sum((predicted.cpu().detach().numpy() == labels.cpu().numpy()).astype(int))

    return (correct / total)

#### Government

In [55]:
## data loaders
BATCH_SIZE = mnli_val.shape[0]

mnli_val_dataset_s1 = MNLI_Dataset(mnli_val_s1_gov_ix, mnli_val_gov_labels)

mnli_val_loader_s1 = torch.utils.data.DataLoader(dataset=mnli_val_dataset_s1,
                                             batch_size=BATCH_SIZE,
                                             collate_fn=mnli_func,
                                             shuffle=False)

mnli_val_dataset_s2 = MNLI_Dataset(mnli_val_s2_gov_ix, mnli_val_gov_labels)

mnli_val_loader_s2 = torch.utils.data.DataLoader(dataset=mnli_val_dataset_s2,
                                             batch_size=BATCH_SIZE,
                                             collate_fn=mnli_func,
                                             shuffle=False)

test_accuracy = test_model(mnli_val_loader_s1,
                           mnli_val_loader_s2,
                           model)

print ("Government genre test accuracy is "+ str(test_accuracy))

labels = tensor([2, 0, 1,  ..., 0, 0, 1], device='cuda:0')
predicted = tensor([2, 1, 2,  ..., 0, 0, 2], device='cuda:0')
Government genre test accuracy is 0.45866141732283466


/home/adc563/pytorch-cpu/py3.6.3/lib/python3.6/site-packages/ipykernel_launcher.py:140: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [66]:
print ([id2token_wiki[token] for token in np.array([*mnli_val_loader_s1][0][0][1])][:20])
print ([id2token_wiki[token] for token in np.array([*mnli_val_loader_s2][0][0][1])][:20])

['IDPA', "'s", 'OIG', "'s", 'mission', 'is', 'to', 'prevent', ',', 'detect', ',', 'and', 'eliminate', 'fraud', ',', 'waste', ',', 'abuse', ',', 'and']
['IDPA', "'s", 'OIG', "'s", 'mission', 'is', 'clear', 'and', 'cares', 'about', 'payment', 'programs', '.', '\x01', '\x01', '\x01', '\x01', '\x01', '\x01', '\x01']


#### Telephone

In [67]:
## data loaders
BATCH_SIZE = mnli_val.shape[0]

mnli_val_dataset_s1 = MNLI_Dataset(mnli_val_s1_tel_ix, mnli_val_tel_labels)

mnli_val_loader_s1 = torch.utils.data.DataLoader(dataset=mnli_val_dataset_s1,
                                             batch_size=BATCH_SIZE,
                                             collate_fn=mnli_func,
                                             shuffle=False)

mnli_val_dataset_s2 = MNLI_Dataset(mnli_val_s2_tel_ix, mnli_val_tel_labels)

mnli_val_loader_s2 = torch.utils.data.DataLoader(dataset=mnli_val_dataset_s2,
                                             batch_size=BATCH_SIZE,
                                             collate_fn=mnli_func,
                                             shuffle=False)

test_accuracy = test_model(mnli_val_loader_s1,
                           mnli_val_loader_s2,
                           model)

print ("Telephone genre test accuracy is "+ str(test_accuracy))

labels = tensor([2, 0, 1,  ..., 1, 0, 2], device='cuda:0')
predicted = tensor([0, 1, 0,  ..., 1, 2, 0], device='cuda:0')
Telephone genre test accuracy is 0.4507462686567164


/home/adc563/pytorch-cpu/py3.6.3/lib/python3.6/site-packages/ipykernel_launcher.py:140: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [68]:
print ([id2token_wiki[token] for token in np.array([*mnli_val_loader_s1][0][0][-1])][:20])
print ([id2token_wiki[token] for token in np.array([*mnli_val_loader_s2][0][0][-1])][:20])

['because', 'like', 'Tech', 'is', 'known', 'to', 'be', 'a', 'good', 'engineering', 'school', 'and', 'A', 'and', 'M', 'maybe', 'is', 'known', 'more', 'for']
['A', 'and', 'M', "'s", 'computer', 'department', 'is', 'unk', 'very', 'well', 'regarded', '.', '\x01', '\x01', '\x01', '\x01', '\x01', '\x01', '\x01', '\x01']


#### Slate

In [77]:
## data loaders
BATCH_SIZE = mnli_val.shape[0]

mnli_val_dataset_s1 = MNLI_Dataset(mnli_val_s1_slate_ix, mnli_val_slate_labels)

mnli_val_loader_s1 = torch.utils.data.DataLoader(dataset=mnli_val_dataset_s1,
                                             batch_size=BATCH_SIZE,
                                             collate_fn=mnli_func,
                                             shuffle=False)

mnli_val_dataset_s2 = MNLI_Dataset(mnli_val_s2_slate_ix, mnli_val_slate_labels)

mnli_val_loader_s2 = torch.utils.data.DataLoader(dataset=mnli_val_dataset_s2,
                                             batch_size=BATCH_SIZE,
                                             collate_fn=mnli_func,
                                             shuffle=False)

test_accuracy = test_model(mnli_val_loader_s1,
                           mnli_val_loader_s2,
                           model)

print ("Slate genre test accuracy is "+ str(test_accuracy))

labels = [0, 1, 2, 0, 2, 2, 0, 1, 0, 1, 0, 0, 2, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 2, 2, 1, 2, 1, 1, 2, 1, 2, 0, 1, 1, 0, 1, 2, 2, 0, 1, 2, 1, 1, 1, 1, 1, 0, 1, 2, 1, 0, 0, 0, 2, 0, 0, 1, 2, 1, 1, 0, 1, 1, 1, 2, 0, 0, 0, 1, 0, 2, 2, 0, 2, 0, 1, 2, 2, 0, 2, 0, 1, 0, 1, 2, 0, 0, 0, 0, 1, 2, 1, 2, 2, 2, 0, 1, 0, 0, 2, 0, 1, 0, 1, 2, 1, 2, 1, 0, 1, 1, 0, 0, 2, 2, 1, 2, 1, 0, 1, 0, 2, 2, 2, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 2, 1, 0, 2, 1, 0, 2, 1, 2, 0, 0, 1, 1, 1, 2, 1, 0, 1, 1, 2, 0, 0, 2, 1, 1, 2, 2, 0, 1, 1, 2, 1, 1, 0, 1, 1, 2, 0, 0, 0, 2, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 0, 2, 2, 0, 0, 1, 1, 2, 0, 1, 1, 0, 0, 1, 1, 2, 2, 1, 2, 1, 1, 0, 0, 2, 1, 1, 2, 0, 2, 1, 1, 0, 2, 1, 0, 0, 0, 1, 2, 2, 2, 2, 0, 1, 0, 0, 2, 2, 1, 2, 0, 0, 2, 2, 1, 1, 1, 1, 1, 1, 0, 2, 0, 1, 0, 0, 2, 1, 0, 2, 1, 0, 0, 0, 0, 2, 1, 0, 0, 0, 2, 2, 0, 0, 1, 0, 1, 0, 2, 0, 1, 2, 1, 1, 0, 0, 2, 0, 0, 2, 1, 2, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 2, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 2, 0, 2, 0, 2, 0, 2, 0, 1, 1, 0, 1, 1, 1, 2, 2, 

/home/adc563/pytorch-cpu/py3.6.3/lib/python3.6/site-packages/ipykernel_launcher.py:140: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [78]:
print ([id2token_wiki[token] for token in np.array([*mnli_val_loader_s1][0][0][5])][:20])
print ([id2token_wiki[token] for token in np.array([*mnli_val_loader_s2][0][0][5])][:20])

['A', 'button', 'on', 'the', 'Chatterbox', 'page', 'will', 'make', 'this', 'easy', ',', 'so', 'please', 'do', 'join', 'in', '.', '\x01', '\x01', '\x01']
['They', 'had', 'to', 'submit', 'a', 'written', 'request', 'before', 'being', 'accepted', '.', '\x01', '\x01', '\x01', '\x01', '\x01', '\x01', '\x01', '\x01', '\x01']


#### Fiction

In [79]:
## data loaders
BATCH_SIZE = mnli_val.shape[0]

mnli_val_dataset_s1 = MNLI_Dataset(mnli_val_s1_fiction_ix, mnli_val_fiction_labels)

mnli_val_loader_s1 = torch.utils.data.DataLoader(dataset=mnli_val_dataset_s1,
                                             batch_size=BATCH_SIZE,
                                             collate_fn=mnli_func,
                                             shuffle=False)

mnli_val_dataset_s2 = MNLI_Dataset(mnli_val_s2_fiction_ix, mnli_val_fiction_labels)

mnli_val_loader_s2 = torch.utils.data.DataLoader(dataset=mnli_val_dataset_s2,
                                             batch_size=BATCH_SIZE,
                                             collate_fn=mnli_func,
                                             shuffle=False)

test_accuracy = test_model(mnli_val_loader_s1,
                           mnli_val_loader_s2,
                           model)

print ("Fiction genre test accuracy is "+ str(test_accuracy))

labels = [0, 2, 1, 2, 1, 1, 2, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 1, 2, 0, 1, 1, 1, 2, 0, 2, 2, 1, 0, 0, 2, 0, 1, 2, 2, 0, 0, 1, 2, 1, 0, 2, 2, 2, 1, 1, 2, 1, 1, 2, 2, 1, 2, 0, 2, 0, 2, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 2, 0, 1, 2, 0, 2, 1, 2, 1, 0, 2, 2, 0, 2, 0, 2, 0, 1, 1, 1, 2, 2, 0, 0, 1, 0, 1, 0, 2, 2, 0, 2, 0, 2, 2, 2, 1, 0, 1, 2, 0, 2, 2, 1, 0, 1, 0, 2, 0, 2, 1, 1, 0, 0, 0, 0, 0, 0, 2, 2, 0, 1, 2, 2, 0, 0, 2, 2, 1, 0, 2, 2, 1, 0, 2, 1, 0, 1, 1, 0, 2, 0, 0, 2, 2, 0, 0, 2, 0, 2, 0, 0, 0, 2, 2, 2, 2, 0, 1, 0, 1, 0, 0, 1, 1, 2, 1, 0, 1, 2, 0, 1, 0, 0, 1, 0, 1, 2, 0, 0, 1, 2, 2, 1, 2, 0, 1, 1, 0, 2, 2, 2, 2, 0, 2, 2, 0, 2, 0, 0, 1, 0, 1, 0, 2, 2, 1, 2, 1, 2, 2, 2, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 2, 2, 1, 2, 1, 1, 1, 0, 1, 2, 1, 1, 1, 2, 2, 2, 0, 1, 0, 2, 2, 0, 0, 1, 2, 2, 2, 0, 2, 0, 0, 0, 2, 0, 2, 1, 1, 2, 2, 1, 1, 0, 0, 0, 2, 0, 2, 0, 1, 1, 1, 2, 2, 2, 2, 1, 2, 2, 0, 0, 0, 1, 1, 2, 1, 1, 0, 1, 0, 2, 2, 0, 2, 2, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 2, 0, 0, 2, 1, 2, 1, 1, 1, 

/home/adc563/pytorch-cpu/py3.6.3/lib/python3.6/site-packages/ipykernel_launcher.py:140: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [81]:
print ([id2token_wiki[token] for token in np.array([*mnli_val_loader_s1][0][0][5])][:30])
print ([id2token_wiki[token] for token in np.array([*mnli_val_loader_s2][0][0][5])][:20])

['And', 'if', 'they', 'did', 'come', ',', 'as', 'remote', 'as', 'that', 'is', ',', 'you', 'and', 'your', 'men', 'look', 'strong', 'enough', 'to', 'handle', 'anything', '.', '\x01', '\x01', '\x01', '\x01', '\x01', '\x01', '\x01']
['The', 'men', 'were', 'warriors', '.', '\x01', '\x01', '\x01', '\x01', '\x01', '\x01', '\x01', '\x01', '\x01', '\x01', '\x01', '\x01', '\x01', '\x01', '\x01']


#### Travel

In [82]:
## data loaders
BATCH_SIZE = mnli_val.shape[0]

mnli_val_dataset_s1 = MNLI_Dataset(mnli_val_s1_travel_ix, mnli_val_travel_labels)

mnli_val_loader_s1 = torch.utils.data.DataLoader(dataset=mnli_val_dataset_s1,
                                             batch_size=BATCH_SIZE,
                                             collate_fn=mnli_func,
                                             shuffle=False)

mnli_val_dataset_s2 = MNLI_Dataset(mnli_val_s2_travel_ix, mnli_val_travel_labels)

mnli_val_loader_s2 = torch.utils.data.DataLoader(dataset=mnli_val_dataset_s2,
                                             batch_size=BATCH_SIZE,
                                             collate_fn=mnli_func,
                                             shuffle=False)

test_accuracy = test_model(mnli_val_loader_s1,
                           mnli_val_loader_s2,
                           model)

print ("Travel genre test accuracy is "+ str(test_accuracy))

labels = [1, 1, 2, 2, 0, 0, 2, 0, 2, 0, 1, 0, 1, 0, 1, 2, 2, 1, 1, 0, 2, 0, 0, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 2, 2, 1, 0, 2, 1, 0, 2, 2, 1, 0, 2, 0, 2, 1, 2, 2, 1, 2, 0, 1, 0, 2, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 2, 0, 0, 2, 0, 0, 1, 0, 1, 0, 2, 1, 2, 0, 2, 0, 1, 0, 0, 2, 1, 2, 2, 1, 2, 2, 2, 2, 0, 1, 0, 2, 1, 1, 1, 1, 0, 0, 2, 1, 2, 1, 1, 2, 2, 1, 0, 1, 0, 0, 2, 1, 0, 2, 1, 2, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 2, 2, 2, 1, 2, 2, 2, 1, 0, 0, 0, 1, 2, 0, 1, 0, 0, 0, 2, 1, 0, 1, 2, 0, 0, 0, 2, 2, 2, 1, 2, 0, 0, 1, 0, 2, 2, 2, 0, 0, 0, 2, 2, 0, 1, 0, 1, 2, 2, 1, 1, 1, 0, 2, 0, 2, 0, 0, 0, 1, 2, 1, 0, 0, 0, 2, 1, 1, 0, 0, 1, 2, 1, 2, 0, 0, 2, 1, 2, 1, 2, 2, 2, 0, 0, 2, 2, 0, 1, 1, 2, 2, 1, 0, 1, 0, 0, 2, 2, 1, 2, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 2, 0, 1, 0, 1, 2, 0, 2, 1, 1, 0, 0, 2, 0, 2, 0, 1, 0, 1, 0, 2, 0, 1, 1, 1, 2, 0, 0, 0, 2, 0, 2, 2, 0, 0, 0, 1, 2, 2, 2, 0, 1, 1, 2, 0, 0, 0, 2, 1, 2, 1, 0, 2, 0, 1, 2, 2, 0, 1, 0, 0, 0, 1, 1, 2, 0, 

/home/adc563/pytorch-cpu/py3.6.3/lib/python3.6/site-packages/ipykernel_launcher.py:140: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [83]:
print ([id2token_wiki[token] for token in np.array([*mnli_val_loader_s1][0][0][2])][:30])
print ([id2token_wiki[token] for token in np.array([*mnli_val_loader_s2][0][0][2])][:20])

['In', 'this', 'enclosed', 'but', 'airy', 'building', ',', 'you', "'ll", 'find', 'ladies', 'with', 'large', 'machetes', 'expertly', 'chopping', 'off', 'hunks', 'of', 'kingfish', ',', 'tuna', ',', 'or', 'shark', 'for', 'eager', 'buyers', '.', '\x01']
['You', "'ll", 'find', 'small', 'lepers', 'chopping', 'of', 'chunks', 'of', 'tuna', ',', 'its', 'the', 'only', 'place', 'they', 'can', 'work', '.', '\x01']
